In [1]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [2]:
!pip install pytorchcv

In [3]:
IDS2017URL = "/gdrive/MyDrive/IDS2017/IDS2017.csv"

In [4]:
import os
os.chdir("/gdrive/My Drive/NBDT_env")

In [5]:
!git clone https://github.com/alvinwan/neural-backed-decision-trees.git

fatal: destination path 'neural-backed-decision-trees' already exists and is not an empty directory.


In [6]:
import os 
os.chdir("/gdrive/My Drive/NBDT_env/neural-backed-decision-trees")

In [7]:
import torch
import pandas as pd
import tensorflow as tf
import numpy as np
import torch.nn as nn
import random

from numpy import vstack
from numpy import argmax
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from torch import Tensor
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Softmax
from torch.nn import Module
from torch.optim import SGD
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_
from torchvision import transforms

In [8]:
class MLP(nn.Module):
    # define model elements
    def __init__(self, n_inputs):
        super(MLP, self).__init__()
        # input to first hidden layer
        self.hidden1 = Linear(n_inputs, 20)
        kaiming_uniform_(self.hidden1.weight, nonlinearity='relu')
        self.act1 = ReLU()
        # second hidden layer
        self.hidden2 = Linear(20, 20)
        kaiming_uniform_(self.hidden2.weight, nonlinearity='relu')
        self.act2 = ReLU()
        # third hidden layer and output
        self.hidden3 = Linear(20, 13)
        xavier_uniform_(self.hidden3.weight)
        self.act3 = Softmax(dim=1)

    def forward(self, X):
        # input to first hidden layer
        X = self.hidden1(X)
        X = self.act1(X)

        # second hidden layer
        X = self.hidden2(X)
        X = self.act2(X)

        # output layer
        X = self.hidden3(X)
        X = self.act3(X)

        return X

In [9]:
model19 = MLP(19)

load model state_dict()

In [10]:
model19.load_state_dict(torch.load('/gdrive/MyDrive/IDS2017/feature19_classes13_lr001_batch10000.pth'))

<All keys matched successfully>

#get wnid from txt

In [11]:
#generate_fname,
import pandas as pd
import numpy as np
import tensorflow as tf
from nbdt.thirdparty.nx import write_graph, get_root, get_depth, read_graph, get_leaves
from nbdt.thirdparty.wn import get_wnids_from_dataset, get_wnids,  wnid_to_name
from nbdt.graph import build_minimal_wordnet_graph, build_random_graph, \
    prune_single_successor_nodes,  \
    get_parser,  get_directory, get_graph_path_from_args, \
    augment_graph, build_induced_graph,  \
    get_roots, synset_to_wnid
from nbdt.hierarchy import print_graph_stats, assert_all_wnids_in_graph
from nbdt.utils import DATASETS, METHODS, Colors, fwd
from nbdt.hierarchy import generate_hierarchy_vis,generate_hierarchy_vis_from

import torch
import os


not enough values to unpack (expected 2, got 0)


In [12]:
wnids = get_wnids_from_dataset('IDS2017')

In [13]:
wnids

['f05933834',
 'f03497657',
 'f05254795',
 'f03441112',
 'f04356056',
 'f04453666',
 'f03236735',
 'f03057021',
 'f04254777',
 'f03605598',
 'f02854739',
 'f04231272',
 'f05563770']

In [14]:
G = build_induced_graph(wnids,
            dataset='IDS2017',
            checkpoint=None,
            model=None,
            linkage="ward",
            affinity="euclidean",
            branching_factor=2,
            state_dict=model19.state_dict())

In [15]:
print_graph_stats(G, 'matched')

[matched] 	 Nodes: 25 	 Depth: 9 	 Max Children: 9


In [16]:
assert_all_wnids_in_graph(G, wnids)

In [17]:
no_prune=False

In [18]:
if not no_prune:
    G = prune_single_successor_nodes(G)
    print_graph_stats(G, 'pruned')
    assert_all_wnids_in_graph(G, wnids)

[pruned] 	 Nodes: 25 	 Depth: 9 	 Max Children: 9


In [19]:
path = get_graph_path_from_args(
    dataset="IDS2017",
    method='induced',
    seed=0,
    branching_factor=2,
    extra=0,
    no_prune=False,
    fname='',
    path='',
    single_path=False,
    induced_linkage='ward',
    induced_affinity='euclidean',
    checkpoint=None,
    arch='MLP19'
)
write_graph(G, path)

In [20]:
path

'./nbdt/hierarchies/IDS2017/graph-induced-MLP19.json'

In [21]:
from nbdt.graph import get_parser

parser = get_parser()
args = parser.parse_args('--vis-sublabels --vis-zoom=1.25 --dataset=IDS2017 --arch=MLP19'.split())

generate_hierarchy_vis(args)

Colors.green('==> Wrote tree to {}'.format(path))

==> Reading from ./nbdt/hierarchies/IDS2017/graph-induced-MLP19.json
Found just 1 root.
==> Wrote HTML to ./IDS2017-induced-MLP19.html 
==> Wrote tree to ./nbdt/hierarchies/IDS2017/graph-induced-MLP19.json 


In [22]:
#G = read_graph(path)
# path_html = "./nbdt/hierarchies/IDS2017/graph-induced-MLP19-noprune.html"
# generate_hierarchy_vis_from(G,None,path_html,verbose=True)
# Colors.green('==> Wrote tree to {}'.format(path))

first stup
creat a graph ./nbdt/hierarchies/IDS2017/graph-induced-MLP19-noprune.json 

step 2


read data
select features
combine lable

In [23]:
df = pd.read_csv(IDS2017URL)
df.loc[df['Label'].isin([ 'Web Attack - Sql Injection','Web Attack - XSS', 'Web Attack - Brute Force']), 'Label'] = 'Web Attack'
df = df[['Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Flow IAT Max', 'Bwd Packet Length Std', 'Fwd Packet Length Max', 'Flow Bytes/s', 'Total Length of Bwd Packets', 'Fwd Packet Length Mean', 'Flow Duration', 'Flow IAT Min', 'Total Length of Fwd Packets', 'Flow IAT Mean', 'Total Backward Packets', 'Bwd Packet Length Max', 'Flow Packets/s', 'Flow IAT Std', 'Fwd IAT Total', 'Bwd Packet Length Min', 'Fwd Packet Length Min', 'Label']]

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (84) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


prepare data

In [24]:
df = df.replace([-np.inf, np.inf], np.nan)
df = df.dropna()

In [25]:
X = df.iloc[:, :-1].apply(lambda x: (x-np.mean(x))/np.std(x)).values
y = df.values[:, -1]
X = X.astype('float32')
le = LabelEncoder()
le.fit(["BENIGN", "DoS Hulk", "PortScan", "DDoS", "DoS GoldenEye","FTP-Patator","SSH-Patator","DoS slowloris","DoS Slowhttptest","Web Attack","Bot","Infiltration","Heartbleed"])
y = le.transform(y)

In [26]:
X.shape

(2827876, 19)

In [27]:
y

array([0, 0, 0, ..., 0, 0, 0])

In [28]:
data1 = np.c_[X,y]

In [29]:
tv_rate = 0.33
train_ds, val_ds = random_split(data1, [len(X) - round(tv_rate * len(X)),round(tv_rate * len(X))])

In [30]:
from torch.utils.data import DataLoader
batch_size = 10000

trainloader = DataLoader(train_ds, batch_size, shuffle=True)
testloader = DataLoader(val_ds, batch_size)

In [31]:
import os
import argparse
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms

from nbdt import data, analysis, loss, models, metrics, tree as T
from nbdt.utils import progress_bar, generate_checkpoint_fname, generate_kwargs, Colors
from nbdt.thirdparty.wn import maybe_install_wordnet
from nbdt.models.utils import load_state_dict, make_kwarg_optional
from nbdt.tree import Tree

In [32]:
datasets = data.myData.names
parser = argparse.ArgumentParser(description="PyTorch CIFAR Training")
parser.add_argument(
    "--batch-size", default=512, type=int, help="Batch size used for training"
)
parser.add_argument(
    "--epochs",
    "-e",
    default=200,
    type=int,
    help="By default, lr schedule is scaled accordingly",
)
parser.add_argument("--dataset", default="CIFAR10", choices=datasets)
parser.add_argument(
    "--arch", default="ResNet18", choices=list(models.get_model_choices())
)
parser.add_argument("--lr", default=0.1, type=float, help="learning rate")
parser.add_argument(
    "--resume", "-r", action="store_true", help="resume from checkpoint"
)

# extra general options for main script
parser.add_argument(
    "--path-resume", default="", help="Overrides checkpoint path generation"
)
parser.add_argument(
    "--name", default="", help="Name of experiment. Used for checkpoint filename"
)
parser.add_argument(
    "--pretrained",
    action="store_true",
    help="Download pretrained model. Not all models support this.",
)
parser.add_argument("--eval", help="eval only", action="store_true")
parser.add_argument(
    "--dataset-test",
    choices=datasets,
    help="If not set, automatically set to train dataset",
)
parser.add_argument(
    "--disable-test-eval",
    help="Allows you to run model inference on a test dataset "
    " different from train dataset. Use an anlayzer to define "
    "a metric.",
    action="store_true",
)

# options specific to this project and its dataloaders
parser.add_argument(
    "--loss", choices=loss.names, default=["CrossEntropyLoss"], nargs="+"
)
parser.add_argument("--metric", choices=metrics.names, default="top1")
parser.add_argument(
    "--analysis", choices=analysis.names, help="Run analysis after each epoch"
)

# other dataset, loss or analysis specific options
data.custom.add_arguments(parser)
T.add_arguments(parser)
loss.add_arguments(parser)
analysis.add_arguments(parser)


In [33]:
datasets

('IDS2017',)

In [34]:
args = parser.parse_args("--lr=0.01 --dataset=IDS2017 --arch=MLP19 --hierarchy=induced-MLP19 --pretrained --loss=SoftTreeSupLoss".split())

In [35]:
loss.set_default_values(args)

device = "cuda" if torch.cuda.is_available() else "cpu"
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Data
print("==> Preparing data..")
dataset_train = getattr(data, args.dataset)
dataset_test = getattr(data, args.dataset_test or args.dataset)

==> Preparing data..


In [36]:
dataset_train

nbdt.data.myData.IDS2017

In [37]:
dataset_test

nbdt.data.myData.IDS2017

In [38]:
transform_train = dataset_train.transform_train()
transform_test = dataset_test.transform_val()

In [39]:
dataset_train_kwargs = generate_kwargs(
    args,
    dataset_train,
    name=f"Dataset {dataset_train.__class__.__name__}",
    globals=locals(),
)
dataset_test_kwargs = generate_kwargs(
    args,
    dataset_test,
    name=f"Dataset {dataset_test.__class__.__name__}",
    globals=locals(),
)
trainset = dataset_train(
    **dataset_train_kwargs,
    transform=True,
)
testset = dataset_test(
    **dataset_test_kwargs,
    transform=True,
)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (84) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (84) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [40]:
assert trainset.classes == testset.classes or args.disable_test_eval, (
    trainset.classes,
    testset.classes,
)

In [41]:
trainset.classes

['BENIGN',
 'DoS Hulk',
 'PortScan',
 'DDoS',
 'DoS GoldenEye',
 'FTP-Patator',
 'SSH-Patator',
 'DoS slowloris',
 'DoS Slowhttptest',
 'Bot']

In [42]:
trainloader = torch.utils.data.DataLoader(
        trainset, batch_size=args.batch_size, shuffle=True, num_workers=2
    )
testloader = torch.utils.data.DataLoader(
        testset, batch_size=100, shuffle=False, num_workers=2
    )


In [43]:
model = getattr(models, args.arch)

In [44]:
net = model()

In [48]:
args

Namespace(analysis=None, arch='MLP19', batch_size=512, dataset='IDS2017', dataset_test=None, disable_test_eval=False, epochs=200, eval=False, exclude_labels=None, hierarchy='induced-MLP19', include_classes=None, include_labels=None, loss=['SoftTreeSupLoss'], lr=0.01, metric='top1', name='', path_graph='/gdrive/MyDrive/NBDT_env/neural-backed-decision-trees/nbdt/hierarchies/IDS2017/graph-induced-MLP19.json', path_resume='', path_wnids='/gdrive/MyDrive/NBDT_env/neural-backed-decision-trees/nbdt/wnids/IDS2017.txt', pretrained=True, probability_labels=None, resume=False, save_k=20, superclass_wnids=None, tree_start_epochs=None, tree_supervision_weight=1, tree_supervision_weight_end=None, tree_supervision_weight_power=None, tree_update_end_epochs=None, tree_update_every_epochs=None, visualize_decision_node_wnid=None, xent_weight=None, xent_weight_end=None, xent_weight_power=None)

In [51]:
tree = Tree.create_from_args(args, classes=trainset.classes)

IndexError: ignored

In [50]:
criterion = None
for _loss in args.loss:
    if criterion is None and not hasattr(nn, _loss):
        criterion = nn.CrossEntropyLoss()
    class_criterion = getattr(loss, _loss)
    loss_kwargs = generate_kwargs(
        args,
        class_criterion,
        name=f"Loss {args.loss}",
        globals=locals(),
    )
criterion = class_criterion(**loss_kwargs)



classes:	(callable) 
criterion:	(callable) 
dataset:	(callable) 
hierarchy:	induced-MLP19 
path_graph:	/gdrive/MyDrive/NBDT_env/neural-backed-decision-trees/nbdt/hierarchies/IDS2017/graph-induced-MLP19.json 
path_wnids:	/gdrive/MyDrive/NBDT_env/neural-backed-decision-trees/nbdt/wnids/IDS2017.txt 


TypeError: ignored